# Commutativity checker for fermionic excitations in second quantization

The excitations are not dependent on the molecule in question. They can be generated by specifying 

1. the number of spatial orbitals: `num_spatial`
2. the number of alpha and beta spins: `num_particles = (num_alpha, num_beta)`

For example, for the $H_2$ molecule, there are 2 spatial orbitals, one $\alpha$-spin particle (spin-up), and one $\beta$-spin particle (spin-down). Therefore, the parameters look like follows:

`num_spatial = 2`

`num_particles = (1, 1)`

In [28]:
from qiskit_nature.second_q.operators import FermionicOp
from qiskit_nature.second_q.circuit.library.ansatzes.utils import generate_fermionic_excitations
from qiskit_nature.second_q.mappers import JordanWignerMapper
import numpy as np
from IPython.display import display, Math

# --- Helper function for clean printing ---
def print_clean_pauli(op):
    """Prints coefficients and Pauli strings line by line."""
    for pauli, coeff in zip(op.paulis, op.coeffs):
        if np.isclose(coeff.imag, 0.0):
            c_str = f"{coeff.real:.4f}"
        elif np.isclose(coeff.real, 0.0):
            c_str = f"{coeff.imag:.4f}j"
        else:
            c_str = f"{coeff:.4f}"
        print(f"  {c_str:>10} * {pauli}")

# 1. SETUP PARAMETERS (MODIFY TO YOUR NEEDS)
num_spatial = 2
num_particles = (1, 1)
num_spin_orbitals = num_spatial * 2    # don't modify

mapper = JordanWignerMapper()

# 2. Get excitations
raw_excitations = []
for rank in [1, 2]:
    raw_excitations += generate_fermionic_excitations(
        num_excitations=rank, 
        num_spatial_orbitals=num_spatial, 
        num_particles=num_particles,
        alpha_spin=True,
        beta_spin=True
    )

print(f"Generated {len(raw_excitations)} raw excitations.\n")

# 3. Build generators
pauli_generators = []
labels = []

display(Math(r"\text{--- 1. Generators and Pauli decomposition ---}"))

for i, exc in enumerate(raw_excitations):
    occ_indices = exc[0]
    vir_indices = exc[1]

    # --- Build The forward term (T) ---
    # Creation on virtual orbitals, annihilation on occupied orbitals
    forward_creation = " ".join([f"a^\\dagger_{{{idx}}}" for idx in vir_indices])
    forward_annihilation = " ".join([f"a_{{{idx}}}" for idx in occ_indices])
    
    # --- Build The conjugate term (T_dagger) ---
    # To find the Hermitian conjugate:
    # 1. Swap creation <-> annihilation
    # 2. Swap indices (occupied <-> virtual)
    # 3. Reverse the order ( (AB)^dagger = B^dagger A^dagger )
    
    # Conjugate of the annihilation part becomes creation (reversed)
    # e.g. (a_0 a_1)^dagger -> a^dagger_1 a^dagger_0
    hc_creation = " ".join([f"a^\\dagger_{{{idx}}}" for idx in reversed(occ_indices)])
    
    # Conjugate of the creation part becomes annihilation (reversed)
    # e.g. (a^dagger_2 a^dagger_3)^dagger -> a_3 a_2
    hc_annihilation = " ".join([f"a_{{{idx}}}" for idx in reversed(vir_indices)])

    # Construct full label
    latex_label = f"G_{{{i}}} = {forward_creation} {forward_annihilation} - {hc_creation} {hc_annihilation}"
    labels.append(latex_label)

    # --- Build operator object ---
    creation_part = " ".join([f"+_{idx}" for idx in vir_indices])
    annihilation_part = " ".join([f"-_{idx}" for idx in occ_indices])
    op_str = f"{creation_part} {annihilation_part}"
    
    T_op = FermionicOp({op_str: 1.0}, num_spin_orbitals=num_spin_orbitals)
    G_op = T_op - T_op.adjoint()
    
    qubit_op = mapper.map(G_op).simplify()
    pauli_generators.append(qubit_op)
    
    # Render
    display(Math(latex_label))
    print_clean_pauli(qubit_op)
    print("-" * 30)

# 4. Check commutation
display(Math(r"\text{--- 2. Non-commuting pairs ---}"))
non_commuting_pairs = 0

for i in range(len(pauli_generators)):
    for j in range(i + 1, len(pauli_generators)):
        op_i = pauli_generators[i]
        op_j = pauli_generators[j]
        
        commutator = (op_i @ op_j) - (op_j @ op_i)
        commutator = commutator.simplify()
        
        if not np.all(np.isclose(commutator.coeffs, 0.0)):
            # Render the pair label
            comm_latex = f"[{labels[i]}, \\quad {labels[j]}] \\neq 0"
            display(Math(comm_latex))
            
            print("Residual Pauli terms:")
            print_clean_pauli(commutator)
            print("-" * 30)
            
            non_commuting_pairs += 1

print(f"\nSummary: Found {non_commuting_pairs} non-commuting pairs out of {len(raw_excitations)*(len(raw_excitations)-1)//2} total pairs.")

Generated 3 raw excitations.



<IPython.core.display.Math object>

<IPython.core.display.Math object>

     0.5000j * IIXY
    -0.5000j * IIYX
------------------------------


<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Residual Pauli terms:
    -0.2500j * XYIZ
     0.2500j * YXIZ
     0.2500j * XYZI
    -0.2500j * YXZI
------------------------------


<IPython.core.display.Math object>

Residual Pauli terms:
    -0.2500j * IZXY
     0.2500j * ZIXY
     0.2500j * IZYX
    -0.2500j * ZIYX
------------------------------

Summary: Found 2 non-commuting pairs out of 3 total pairs.


In [13]:
%pip list | grep qiskit

qiskit                  2.2.3
qiskit-aer              0.17.2
qiskit-algorithms       0.4.0
qiskit-ibm-runtime      0.44.0
qiskit-nature           0.7.2
Note: you may need to restart the kernel to use updated packages.


<div style="background-color:#f0f0f0; padding:10px; border-radius:5px;">
This code is a part of Quantum AI Biomedical Research Lab project 
    
`Estimating molecular ground and excited state energies on quantum computers'
    
© Copyright Renata Wong, 2026.

This code is licensed under the CC BY-NC 4.0 License. You may
obtain a copy of this license in the LICENSE.txt file in the root directory
of this source tree or at https://creativecommons.org/licenses/by-nc/4.0/deed.en.

Any modifications or derivative works of this code must retain this
copyright notice, and modified files need to carry a notice indicating
that they have been altered from the originals.
</div>

<div style="background-color:#f0f0f0; padding:10px; border-radius:5px;">
This work was supported by the National Science and Technology Council (Taiwan) grant No. NSTC 114-2112-M-182-002-MY3 and Chang Gung Memorial Hospital grant No. BMRPL94.

This work comes with an accompanying paper titled 'Quantum circuit compilation for fermionic excitations using the Jordan-Wigner mapping'.
</div>